In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import sqlite3 as sq
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Occupation:

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Occupation" dataset
+ Select the following dimensions: Region, Occupation  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Census Places"   
+ Under "Occupation" expand "Total-All Occupations" and select the combination of 2 digit SOCS codes and built groups in Jobs EQ as described below, leaving Total as well  

Occupations to Select:  
(SOC Codes)  
+ Total - All Occupations  
+ Healthcare Practitioners and Technical Occupations (29)  
+ Healthcare Support Occupations (31)  
+ Protective Service Occupations (33)  
+ Production Occupations (51)  
+ Transportation and Material Moving Occupations (53)  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Business & Financial (11: Management & 13: Business & Financial Operations)  
+ Computer, Mathematical, & Engineering (15: Computer & Mathematical, 17: Architecture & Engineering, & 19: Life, Physical, & Social Science)  
+ Education, Legal, Social, Arts, & Media (21: Community & Social Service, 23: Legal, 25: Educational Instruction & Library, & 27: Art, Design, Entertainment, Sports, & Media)  
+ Food, Property, & Personal Services (35: Food Preparation & Serving, 37: Building & Grounds Cleaning & Maintenance, & 39: Personal Care & Service)  
+ Sales & Office Support (41: Sales & Related & 43: Office & Administrative Support)  
+ Natural Resources, Construction, & Maintenance (45: Farming Fishing & Forestry, 47: Construction & Extraction, & 49: Installation Maintenance & Repair)  

Then put in draft mode & select the following values: "Employment (Place of Work)", "Employment (Place of Residence)", "Annual Wage (Mean)", "Annual Wage (Median)", "Location Quotient", Demand - Total(Forecast) (3 Years), Demand - Growth Rate (Forecast)(3 Years)". Put region on the left side and occupation at the top.

Once it is completed, there are a few things that are just easier to do in excel. Open and delete the top row that says "Occupation Data", and delete the FIPS column. Rename the Region column to NAME, add NAME above this to create a double header. Delete the data notes at the bottom. Unmerge all of the occupation cells at the top and simply copy/paste the occupation category, ensuring that each column has a group header above the value header. Finally, select all numerical data and format as "general".  

Save in "Data Downloads" as: "JobsEQOccupationalEmploymentforOutlook_AllGeos.csv"

In [40]:
#import data and set a variable keeping track of the most recent download's year and quarter
data = pd.read_csv('../../Data Downloads/JobsEQ_OccupationalEmploymentforOutlook_AllGeos.csv')
recentquarter = 'Q3 2022'
data.head(2)

,NAME,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Practitioners and Technical Occupations (29-0000).1,Healthcare Practitioners and Technical Occupations (29-0000).2,Healthcare Practitioners and Technical Occupations (29-0000).3,Healthcare Practitioners and Technical Occupations (29-0000).4,Healthcare Practitioners and Technical Occupations (29-0000).5,Healthcare Practitioners and Technical Occupations (29-0000).6,Healthcare Support Occupations (31-0000),Healthcare Support Occupations (31-0000).1,Healthcare Support Occupations (31-0000).2,Healthcare Support Occupations (31-0000).3,Healthcare Support Occupations (31-0000).4,Healthcare Support Occupations (31-0000).5,Healthcare Support Occupations (31-0000).6,Protective Service Occupations (33-0000),Protective Service Occupations (33-0000).1,Protective Service Occupations (33-0000).2,Protective Service Occupations (33-0000).3,Protective Service Occupations (33-0000).4,Protective Service Occupations (33-0000).5,Protective Service Occupations (33-0000).6,Production Occupations (51-0000),Production Occupations (51-0000).1,Production Occupations (51-0000).2,Production Occupations (51-0000).3,Production Occupations (51-0000).4,Production Occupations (51-0000).5,Production Occupations (51-0000).6,Transportation and Material Moving Occupations (53-0000),Transportation and Material Moving Occupations (53-0000).1,Transportation and Material Moving Occupations (53-0000).2,Transportation and Material Moving Occupations (53-0000).3,Transportation and Material Moving Occupations (53-0000).4,Transportation and Material Moving Occupations (53-0000).5,Transportation and Material Moving Occupations (53-0000).6,Business & Financial,Business & Financial.1,Business & Financial.2,Business & Financial.3,Business & Financial.4,Business & Financial.5,Business & Financial.6,"Computer, Mathematical, & Engineering","Computer, Mathematical, & Engineering.1","Computer, Mathematical, & Engineering.2","Computer, Mathematical, & Engineering.3","Computer, Mathematical, & Engineering.4","Computer, Mathematical, & Engineering.5","Computer, Mathematical, & Engineering.6","Education, Legal, Social, Arts, & Media","Education, Legal, Social, Arts, & Media.1","Education, Legal, Social, Arts, & Media.2","Education, Legal, Social, Arts, & Media.3","Education, Legal, Social, Arts, & Media.4","Education, Legal, Social, Arts, & Media.5","Education, Legal, Social, Arts, & Media.6","Food, Property, & Personal Services","Food, Property, & Personal Services.1","Food, Property, & Personal Services.2","Food, Property, & Personal Services.3","Food, Property, & Personal Services.4","Food, Property, & Personal Services.5","Food, Property, & Personal Services.6",Sales & Office Support,Sales & Office Support.1,Sales & Office Support.2,Sales & Office Support.3,Sales & Office Support.4,Sales & Office Support.5,Sales & Office Support.6,"Natural Resources, Construction, & Maintenance","Natural Resources, Construction, & Maintenance.1","Natural Resources, Construction, & Maintenance.2","Natural Resources, Construction, & Maintenance.3","Natural Resources, Construction, & Maintenance.4","Natural Resources, Construction, & Maintenance.5","Natural Resources, Construction, & Maintenance.6"
0,NAME,Empl (Place of Work),Empl (Place of Residence),Annual Mean Wage1,Ann 50th %ile Wage1,LQ,Total 3 Year Forecast Demand,3 Year Forecast Growth Rate,Empl (Place of Work),Empl (Place of Residence),Annual Mean Wage1,Ann 50th %ile Wage1,LQ,Total 3 Year Forecast Demand,3 Year Forecast Growth Rate,Empl (Place of Work),Empl (Place of Residence),Annual Mean Wage1,Ann 50th %ile Wage1,LQ,Total 3 Year Forecast Demand,3 Year Forecast Growth Rate,Empl (Place of Work),Empl (Place of Residence),Annual Mean Wage1,Ann 50th %ile Wage1,LQ,Total 3 Year Forecast Demand,3

In [41]:
#set the index as the NAME column, transpose, rename the double NAME to value (recall the double in the download excel formatting), then reset index without dropping
data = data.set_index('NAME').transpose().rename(columns = {'NAME': 'Value'}).reset_index(drop = False)
data.head(2)

NAME,index,Value,USA,Tennessee,GNRC,MPO,GNRC Region,"Castalian Springs CDP, TN","Dickson city, TN","Berry Hill city, TN","Orlinda city, TN","Ashland City town, TN","Burns town, TN","Brentwood city, TN","Goodlettsville city, TN","Cottontown CDP, TN","Fairfield CDP, TN","Bransford CDP, TN","Cumberland City town, TN","New Johnsonville city, TN","Pegram town, TN","Mitchellville city, TN","Mount Pleasant city, TN","Greenbrier town, TN","Shackle Island CDP, TN","Adams city, TN","New Deal CDP, TN","Green Hill CDP, TN","Portland city, TN","McEwen city, TN","Coopertown town, TN","Thompson's Station town, TN","Waverly city, TN","Mount Juliet city, TN","Lebanon city, TN","White House city, TN","Belle Meade city, TN","Cross Plains city, TN","Smyrna town, TN","Ridgetop city, TN","Franklin city, TN","Rural Hill CDP, TN","White Bluff town, TN","Slayden town, TN","Clarksville city, TN","Dover city, TN","Vanleer town, TN","Watertown city, TN","Nolensville town, TN","La Vergne city, TN","Springfield city, TN","Erin city, TN","Rockvale CDP, TN","Columbia city, TN","Kingston Springs town, TN","Spring Hill city, TN","Cedar Hill city, TN","Millersville city, TN","Walterhill CDP, TN","Pleasant View city, TN","Hendersonville city, TN","Graball CDP, TN","Tennessee Ridge town, TN","Christiana CDP, TN","Nashville-Davidson metropolitan government (balance), TN","Oak Hill city, TN","Westmoreland town, TN","Bethpage CDP, TN","Fairview city, TN","Murfreesboro city, TN","Eagleville city, TN","Walnut Grove CDP, TN","Charlotte town, TN","Oak Grove CDP, TN","Forest Hills city, TN","Hartsville/Trousdale County, TN","Gallatin city, TN","Dickson County, Tennessee","Wilson County, Tennessee","Rutherford County, Tennessee","Houston County, Tennessee","Trousdale County, Tennessee","Stewart County, Tennessee","Williamson County, Tennessee","Cheatham County, Tennessee","Robertson County, Tennessee","Sumner County, Tennessee","Montgomery County, Tennessee","Maury County, Tennessee","Humphreys County, Tennessee","Davidson County, Tennessee"
0,Total - All Occupations,Empl (Place of Work),160228834.1,3393465.974,1137806.449,1069206.722,1179321.604,44.95023478,12896.62367,18305.70634,444.4557833,5004.148127,193.9399826,57367.73207,14015.99406,20.48620101,1.13594141,6.924162715,819.5843198,648.5876081,297.5102232,28.11907875,2159.258379,1248.893069,455.9569644,66.71050635,58.35656007,679.7465718,7850.382089,509.3897731,1699.544258,1751.394166,2151.251249,18180.47968,34322.11643,5010.687803,1319.240489,295.2604575,41763.10981,248.4080791,82917.11856,172.1536956,934.5486851,16.61435935,54377.706,994.9018938,52.97700463,559.582245,2768.05407,18353.62288,11299.91752,902.0439145,126.0336023,22583.88945,903.4499074,10403.2217,64.48701561,1396.595166,203.8628607,1602.362328,25331.89858,1.639133467,230.592565,378.0177191,524692.9118,1223.219646,899.1212574,65.71600211,1886.486956,76306.65867,481.4588668,48.77829779,1029.085094,61.28944771,896.8843782,2301.756971,23351.64908,20319.10074,66367.40569,149974.3815,1784.438796,2301.780438,3331.302605,160642.2024,10900.82605,25726.08282,66405.14418,64975.84903,41515.15459,6501.58425,558576.351
1,Total - All Occupations.1,Empl (Place of Residence),155375878.7,3243460.781,1119963.717,1013683.541,1170961.728,366.2643395,7219.625829,737.4130563,471.345686,2335.753263,831.6813738,27146.55374,9121.178198,262.5999222,63.72290808,92.83144627,141.4764065,717.9770539,1025.117682,128.8891405,2538.007874,3799.250934,1868.230795,382.9705216,191.6696147,3711.333729,6120.483768,640.9738102,1951.929638,2052.801694,1544.367505,19610.34235,18492.2354,6638.92584,1418.122073,1269.911169,30896.6606,1045.283988,49972.29839,1623.491425,1925.811198,51.86371816,68455.62579,707.9904326,207.0561371,859.9617037,7346.69608,18062.69356,8423.91391,412.8191253,616.3835405,21540.03966,1461.480979,24574.30118,204.5300297,3496.532392,310.1582281,2420.371842,32881.43818,116.9626374,573.4017372,2675.860518,397872.8859,2715.494349,1197.675433,168.9128072,6273.898099,77785.0511

In [42]:
#strip the index and value columns; remove blank space that would be heading or trailing the strings
data['index'] = data['index'].str.strip()
data['Value'] = data['Value'].str.strip()

In [43]:
#set these strings as the index, so that you can transpose the df and fill the null values with zeros to convert the datatype for calculations
data = data.set_index(['index', 'Value']).transpose().fillna(0)

In [44]:
#make a list of these numerical columns, and convert them to float datatype
cols = data.columns
data[cols] = data[cols].astype(float)

In [45]:
#list occupations and calculate the percent of the total for all (except for total all occupations)
occs = ['Healthcare Practitioners and Technical Occupations (29-0000)', 'Healthcare Support Occupations (31-0000)', 
          'Protective Service Occupations (33-0000)',
          'Production Occupations (51-0000)', 'Transportation and Material Moving Occupations (53-0000)', 'Business & Financial', 
          'Computer, Mathematical, & Engineering', 'Education, Legal, Social, Arts, & Media', 'Food, Property, & Personal Services', 
          'Sales & Office Support', 'Natural Resources, Construction, & Maintenance']
for occ in occs:
    data[('{}'.format(occ), 'Percent of Total')] = percent(data[('{}'.format(occ), 'Empl (Place of Work)')], 
                                                           data[('Total - All Occupations', 'Empl (Place of Work)')])

In [46]:
#transpose this df now that you've made calculations, and reset the index without dropping
data = data.transpose().reset_index(drop = False)

In [47]:
#create a dataframe from the "index" column (occupation) where you split the string on the "." (.1, .2, because of duplicates), expand on delimiter
boop = data['index'].str.split(pat = ".", expand = True)
#set new column data['Occupation'] as the first item in this other dataframe (position 0)
data['Occupation'] = boop[0]

In [48]:
#drop the original index column
data.drop(columns = 'index', inplace = True)

In [49]:
#list the columns of the dataframe, as these will be made into the value variables in a melt function
cols = data.columns
#melt the data (wide to long)
data = data.melt(var_name = 'NAME', id_vars = ['Value', 'Occupation'], value_vars = cols, value_name = 'placeholder')
data.head(2)

,Value,Occupation,NAME,placeholder
0,Empl (Place of Work),Total - All Occupations,USA,160228834.1
1,Empl (Place of Residence),Total - All Occupations,USA,155375878.7


In [50]:
#it will be easiest to get the occupation name up top to rename these, so pivot the table so it's ready to go
data = data.pivot(index = ['Occupation', 'NAME'], columns = 'Value')['placeholder']
#reset the index without dropping, set the index as "Occupation", and transpose to rename
data = data.reset_index(drop = False).set_index('Occupation').transpose()

In [51]:
#rename occupations
data = data.rename(columns = {'Healthcare Practitioners and Technical Occupations (29-0000)': 'Healthcare Practitioners & Technical',
                              'Healthcare Support Occupations (31-0000)': 'Healthcare Support', 
                              'Protective Service Occupations (33-0000)': 'Protective Services',
                              'Production Occupations (51-0000)': 'Production', 
                              'Transportation and Material Moving Occupations (53-0000)': 'Transportation & Material Moving'}).transpose().reset_index(drop = False)

In [ ]:
occs = list(data['Occupation'].unique())

In [52]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos before resetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus).transpose().reset_index()

NAME,"Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Cheatham County, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee",GNRC,GNRC Region,"Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hartsville/Trousdale County, Tennessee","Hendersonville city, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Kingston Springs town, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee",MPO,"Maury County, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Montgomery County, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Orlinda city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Spring Hill city, Tennessee","Springfield city, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee",Tennessee,"Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Trousdale County, Tennessee",United States,"Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Cheatham County, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee",GNRC,GNRC Region,"Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hartsville/Trousdale County, Tennessee","Hendersonville city, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Kingston Springs town, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee",MPO,"Maury County, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Montgomery County, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Orlinda city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Spring Hill city, Tennessee","Springfield city, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee",Tennessee,"Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Trousdale County, Tennessee",United States,"Vanleer town, Tennessee","Watertown

In [38]:
#rename the columns
data = data.rename(columns = {'Total 3 Year Forecast Demand': '3 Year Forecast Demand', 
                              'Empl (Place of Work)': 'Employment', 'LQ': 'Location Quotient', 'Annual Mean Wage1': 'Average Annual Wages', 
                              'Ann 50th %ile Wage1': 'Median Annual Wages'})

In [39]:
#multiply the forecast growth rate *100 to fit the other formatting and drop the demand as we won't use right now
data['3 Year Forecast Growth Rate'] = data['3 Year Forecast Growth Rate'] * 100
#data = data.drop(columns = ['3 Year Forecast Demand', 'Empl (Place of Residence)', 'Average Annual Wages'])

In [24]:
#this will set a year and quarter column to the variable set upon data import
data['Year and Quarter'] = recentquarter

In [127]:
cols = ['3 Year Forecast Growth Rate', 'Median Annual Wages', 'Employment', 'Location Quotient', 'Percent of Total']
data[cols] = data[cols].astype(float)

In [128]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [129]:
#final check
data.head()

Value,NAME,Occupation,3 Year Forecast Growth Rate,Median Annual Wages,Employment,Location Quotient,Percent of Total,Year and Quarter,GEO_ID,Source
0,"Adams city, Tennessee",Business & Financial,4.353381,84800.0,7.603079,0.859852,11.397124,Q3 2022,1600000US4700200,JobsEQ
1,"Ashland City town, Tennessee",Business & Financial,3.087481,79700.0,536.912021,0.809471,10.729339,Q3 2022,1600000US4702180,JobsEQ
2,"Belle Meade city, Tennessee",Business & Financial,5.271411,94400.0,133.751693,0.764898,10.138538,Q3 2022,1600000US4704620,JobsEQ
3,"Berry Hill city, Tennessee",Business & Financial,2.353006,87800.0,2585.023590,1.065385,14.121409,Q3 2022,1600000US4705140,JobsEQ
4,"Brentwood city, Tennessee",Business & Financial,8.704532,89400.0,12406.233410,1.631551,21.625804,Q3 2022,1600000US4708280,JobsEQ


In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NAME                         888 non-null    object 
 1   Occupation                   888 non-null    object 
 2   3 Year Forecast Growth Rate  888 non-null    float64
 3   Median Annual Wages          888 non-null    float64
 4   Employment                   888 non-null    float64
 5   Location Quotient            888 non-null    float64
 6   Percent of Total             814 non-null    float64
 7   Year and Quarter             888 non-null    object 
 8   GEO_ID                       816 non-null    object 
 9   Source                       888 non-null    object 
dtypes: float64(5), object(5)
memory usage: 69.5+ KB


In [131]:
#export to the SQLite database as only the JobsEQ quarterly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_OccupationalEmployment_3YROutlook', conn, if_exists = 'replace', index = False)

888

Industry:  

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ Select the "Industry" dataset, quarterly  
+ Select the following dimensions: Region, Industry  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Census Places"   
+ Under "Industry" expand "Total-All Industries" and select the combination of 2 digit NAICS codes and built groups in Jobs EQ as described below, leaving Total as well  

Industries to Select:  
(NAICS Codes)  
+ Total - All Industries  
+ Construction (23)  
+ Manufacturing (31)  
+ Information (51)  
+ Public Administration (92)  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Other/Unclassified (81: Other, Except Public Administration, 99: Unclassified)  
+ Leisure & Hospitality (71: Arts, Entertainment, & Recreation, 72: Accommodation & Food Services)   
+ Education & Health Services (61: Education, 62: Healthcare & Social Assistance)   
+ Professional & Business Services (54: Professional, Scientific, & Technical, 55: Management of Companies & Enterprises, 56: Administrative & Support & Waste Management & Remediation)   
+ Financial Activities (52: Finance & Insurance, 53: Real Estate & Rental & Leasing)   
+ Trade, Transportation & Utilities (42: Wholesale Trade, 44-45: Retail Trade, 48-49: Transportation & Warehousing, 22: Utilities)  
+ Natural Resources & Mining (11: Agricultural Activities, Forestry, Fishing, & Hunting, 21: Mining, Quarrying, Oil & Gas)  

Go into draft mode and select the following values: "Employment (4Q Avg)", "Avg Ann Wages per Worker", "Location Quotient (4Q Avg)", Forecast Growth Demand (3 Years), Forecast Growth Rate (3 Years)". Have region on the left hand side and industry up top.   

Once it is completed, there are a few things that are just easier to do in excel. Open and delete the top row that says "Industry Data", and delete the FIPS column. Rename the Region column to NAME, add NAME above this to create a double header. Delete the data notes at the bottom. Unmerge all of the industry cells at the top and simply copy/paste the industry category, ensuring that each column has a group header above the value header. Finally, select all numerical data and format as "general". Save as "JobsEQIndustryEmployment_AllGeos.csv"

In [85]:
#import data and set a variable keeping track of the most recent download's year and quarter
data = pd.read_csv('../../Data Downloads/JobsEQ_IndustryEmploymentforOutlook_AllGeos.csv')
recentquarter = 'Q3 2022'
data.head(2)

,NAME,Total - All Industries,Total - All Industries.1,Total - All Industries.2,Total - All Industries.3,Total - All Industries.4,Construction (23),Construction (23).1,Construction (23).2,Construction (23).3,Construction (23).4,Manufacturing (31),Manufacturing (31).1,Manufacturing (31).2,Manufacturing (31).3,Manufacturing (31).4,Public Administration (92),Public Administration (92).1,Public Administration (92).2,Public Administration (92).3,Public Administration (92).4,Other/Unclassified,Other/Unclassified.1,Other/Unclassified.2,Other/Unclassified.3,Other/Unclassified.4,Leisure & Hospitality,Leisure & Hospitality.1,Leisure & Hospitality.2,Leisure & Hospitality.3,Leisure & Hospitality.4,Education & Health Services,Education & Health Services.1,Education & Health Services.2,Education & Health Services.3,Education & Health Services.4,Professional & Business Services,Professional & Business Services.1,Professional & Business Services.2,Professional & Business Services.3,Professional & Business Services.4,Financial Activities,Financial Activities.1,Financial Activities.2,Financial Activities.3,Financial Activities.4,"Trade, Transportation, & Utilities","Trade, Transportation, & Utilities.1","Trade, Transportation, & Utilities.2","Trade, Transportation, & Utilities.3","Trade, Transportation, & Utilities.4",Natural Resources & Mining,Natural Resources & Mining.1,Natural Resources & Mining.2,Natural Resources & Mining.3,Natural Resources & Mining.4,Information (51),Information (51).1,Information (51).2,Information (51).3,Information (51).4
0,NAME,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate
1,USA,160228872.7,68131.51545,1,2613334.463,0.01631001,9518559.788,65968.40292,1,78547.16973,0.008252002,12958908.26,79193.14067,1,-49159.05958,-0.003793457,7396552.084,76160.01961,1,-31218.05675,-0.004220623,7166677.173,41300.1588,1,180149.6628,0.025137125,16620496.59,30889.57919,1,738169.9827,0.044413233,35724748.81,60780.25737,1,1014550.502,0.028399094,24578092.06,90645.43792,1,466468.5793,0.018979039,9306460.336,115800.6019,1,83944.02363,0.009019973,31000554.51,56819.67657,1,16835.61518,0.000543075,2697700.784,59073.71732,1,45554.15296,0.016886288,3260122.324,142954.6235,1,97593.46044,0.029935521


In [86]:
#set the index as the NAME column, transpose, rename the double NAME to value (recall the double in the download excel formatting), then reset index without dropping
data = data.set_index('NAME').transpose().rename(columns = {'NAME': 'Value'}).reset_index(drop = False)
data.head(2)

NAME,index,Value,USA,Tennessee,GNRC,MPO,GNRC Region,"Castalian Springs CDP, TN","Dickson city, TN","Berry Hill city, TN","Orlinda city, TN","Ashland City town, TN","Burns town, TN","Brentwood city, TN","Goodlettsville city, TN","Cottontown CDP, TN","Fairfield CDP, TN","Bransford CDP, TN","Cumberland City town, TN","New Johnsonville city, TN","Pegram town, TN","Mitchellville city, TN","Mount Pleasant city, TN","Greenbrier town, TN","Shackle Island CDP, TN","Adams city, TN","New Deal CDP, TN","Green Hill CDP, TN","Portland city, TN","McEwen city, TN","Coopertown town, TN","Thompson's Station town, TN","Waverly city, TN","Mount Juliet city, TN","Lebanon city, TN","White House city, TN","Belle Meade city, TN","Cross Plains city, TN","Smyrna town, TN","Ridgetop city, TN","Franklin city, TN","Rural Hill CDP, TN","White Bluff town, TN","Slayden town, TN","Clarksville city, TN","Dover city, TN","Vanleer town, TN","Watertown city, TN","Nolensville town, TN","La Vergne city, TN","Springfield city, TN","Erin city, TN","Rockvale CDP, TN","Columbia city, TN","Kingston Springs town, TN","Spring Hill city, TN","Cedar Hill city, TN","Millersville city, TN","Walterhill CDP, TN","Pleasant View city, TN","Hendersonville city, TN","Graball CDP, TN","Tennessee Ridge town, TN","Christiana CDP, TN","Nashville-Davidson metropolitan government (balance), TN","Oak Hill city, TN","Westmoreland town, TN","Bethpage CDP, TN","Fairview city, TN","Murfreesboro city, TN","Eagleville city, TN","Walnut Grove CDP, TN","Charlotte town, TN","Oak Grove CDP, TN","Forest Hills city, TN","Hartsville/Trousdale County, TN","Gallatin city, TN","Dickson County, Tennessee","Wilson County, Tennessee","Rutherford County, Tennessee","Houston County, Tennessee","Trousdale County, Tennessee","Stewart County, Tennessee","Williamson County, Tennessee","Cheatham County, Tennessee","Robertson County, Tennessee","Sumner County, Tennessee","Montgomery County, Tennessee","Maury County, Tennessee","Humphreys County, Tennessee","Davidson County, Tennessee"
0,Total - All Industries,Empl (4Q Mov Avg),160228872.7,3393471.82,1137806.449,1069206.722,1179321.604,44.95023478,12896.62367,18305.70634,444.4557833,5004.148127,193.9399826,57367.73207,14015.99406,20.48620101,1.13594141,6.924162714,819.5843198,648.5876081,297.5102232,28.11907875,2159.258379,1248.893069,455.9569644,66.71050635,58.35656007,679.7465718,7850.382089,509.3897731,1699.544258,1751.394166,2151.251249,18180.47968,34322.11643,5010.687803,1319.240489,295.2604575,41763.10981,248.4080791,82917.11856,172.1536956,934.5486851,16.61435935,54377.706,994.9018938,52.97700463,559.582245,2768.05407,18353.62288,11299.91752,902.0439145,126.0336023,22583.88945,903.4499074,10403.2217,64.48701561,1396.595166,203.8628607,1602.362328,25331.89858,1.639133468,230.592565,378.0177191,524692.9118,1223.219646,899.1212574,65.71600211,1886.486956,76306.65867,481.4588668,48.77829779,1029.085094,61.28944771,896.8843782,2301.756971,23351.64908,20319.10074,66367.40576,149974.3815,1784.438803,2301.780446,3331.302611,160642.2023,10900.82609,25726.08278,66405.14408,64975.84896,41515.15449,6501.584258,558576.3509
1,Total - All Industries.1,Avg Ann Wages per Worker,68131.51545,60031.66511,66927.91274,68493.85151,66465.36753,60086.86948,47136.54224,69946.93106,43638.62558,52602.62878,45853.9044,82659.52444,71855.04852,48986.5281,47810.02877,54854.17044,50401.93671,52383.40499,48201.9717,53821.5374,53427.51876,47233.51893,47127.68825,46817.25117,51278.48322,52346.56625,53244.49331,48184.52185,46107.41973,69533.50188,50295.49653,51205.16243,51401.96555,47779.31079,72434.414,45052.48248,56023.15979,44698.3786,81474.38147,54640.13397,47100.31515,54429.88952,45944.60749,46548.51313,48013.09625,48790.25679,75921.42042,56280.86147,47990.51052,37204.49499,51234.49765,54656.07975,50382.86787,62870.25651,47177.5732,51443.31052,54652.10918,52365.87331,52982.1131,43960.84106,36002.34602,50246.59791,74563.23899,69437.80777,52132.16238,48847.1518,74386.81945,55389.12709,48870.6

In [87]:
#strip the index and value columns; remove blank space that would be heading or trailing the strings
data['index'] = data['index'].str.strip()
data['Value'] = data['Value'].str.strip()

In [88]:
#set these strings as the index, so that you can transpose the df and fill the null values with zeros to convert the datatype for calculations
data = data.set_index(['index', 'Value']).transpose().fillna(0)

In [89]:
#make a list of these numerical columns, and convert them to float datatype
cols = data.columns
data[cols] = data[cols].astype(float)

In [90]:
#list industries and calculate the percent of the total for all (except for total all industries)
inds = ['Construction (23)', 'Manufacturing (31)', 'Information (51)', 'Public Administration (92)', 'Other/Unclassified', 'Leisure & Hospitality', 
        'Education & Health Services', 'Professional & Business Services', 'Financial Activities', 'Trade, Transportation, & Utilities', 'Natural Resources & Mining']
for ind in inds:
    data[('{}'.format(ind), 'Percent of Total')] = percent(data[('{}'.format(ind), 'Empl (4Q Mov Avg)')], data[('Total - All Industries', 'Empl (4Q Mov Avg)')])

In [91]:
#transpose this df now that you've made calculations, and reset the index without dropping
data = data.transpose().reset_index(drop = False)

In [92]:
#create a dataframe from the "index" column (industry) where you split the string on the "." (.1, .2, because of duplicates), expand on delimiter
boop = data['index'].str.split(pat = ".", expand = True)
#set new column data['Industry'] as the first item in this other dataframe (position 0)
data['Industry'] = boop[0]

In [93]:
#drop the original index column
data.drop(columns = 'index', inplace = True)

In [94]:
#list the columns of the dataframe, as these will be made into the value variables in a melt function
cols = data.columns
#melt the data (wide to long)
data = data.melt(var_name = 'NAME', id_vars = ['Value', 'Industry'], value_vars = cols, value_name = 'placeholder')
data.head(2)

,Value,Industry,NAME,placeholder
0,Empl (4Q Mov Avg),Total - All Industries,USA,1.602289e+08
1,Avg Ann Wages per Worker,Total - All Industries,USA,6.813152e+04


In [95]:
#it will be easiest to get the industry name up top to rename these, so pivot the table so it's ready to go
data = data.pivot(index = ['Industry', 'NAME'], columns = 'Value')['placeholder']
#reset the index without dropping, set the index as "Industry", and transpose to rename
data = data.reset_index(drop = False).set_index('Industry').transpose()

In [96]:
#rename industries
data = data.rename(columns = {'Construction (23)': 'Construction', 'Information (51)': 'Information', 'Manufacturing (31)': 'Manufacturing', 
                              'Public Administration (92)': 'Public Administration', 'Total - All Industries': 'Total'}).transpose().reset_index(drop = False)

In [97]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos before resetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus).transpose().reset_index()

In [98]:
#rename the columns
data = data.rename(columns = {'3 Year Forecast Growth Demand': '3 Year Forecast Demand', 'Avg Ann Wages per Worker': 'Average Annual Wages', 
                              'Empl (4Q Mov Avg)': 'Employment', 'LQ (4Q Mov Avg)': 'Location Quotient'})

In [99]:
#multiply the forecast growth rate *100 to fit the other formatting and drop the demand as we won't use right now
data['3 Year Forecast Growth Rate'] = data['3 Year Forecast Growth Rate'] * 100
data = data.drop(columns = '3 Year Forecast Demand')

In [100]:
#this will set a year and quarter column to the variable set upon data import
data['Year and Quarter'] = recentquarter

In [101]:
#remove geography names that contain CDP, make sure that the index has no name for feather format export, and drop the index to have standard index format for feather
data = data.rename_axis(None, axis=1).reset_index(drop = True)

In [102]:
cols = ['3 Year Forecast Growth Rate', 'Average Annual Wages', 'Employment', 'Location Quotient', 'Percent of Total']
data[cols] = data[cols].astype(float)

In [103]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [104]:
#final check
data.head()

,NAME,Industry,3 Year Forecast Growth Rate,Average Annual Wages,Employment,Location Quotient,Percent of Total,Year and Quarter,GEO_ID,Source
0,"Adams city, Tennessee",Construction,3.246611,0.00000,0.328852,0.082981,0.492955,Q3 2022,1600000US4700200,JobsEQ
1,"Ashland City town, Tennessee",Construction,2.460413,56502.15272,182.180862,0.612833,3.640597,Q3 2022,1600000US4702180,JobsEQ
2,"Belle Meade city, Tennessee",Construction,2.264722,75583.43544,26.515453,0.338333,2.009903,Q3 2022,1600000US4704620,JobsEQ
3,"Berry Hill city, Tennessee",Construction,2.286419,75980.42524,1005.285343,0.924427,5.491650,Q3 2022,1600000US4705140,JobsEQ
4,"Brentwood city, Tennessee",Construction,6.878503,84212.01593,2380.604373,0.698536,4.149727,Q3 2022,1600000US4708280,JobsEQ


In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NAME                         888 non-null    object 
 1   Industry                     888 non-null    object 
 2   3 Year Forecast Growth Rate  888 non-null    float64
 3   Average Annual Wages         888 non-null    float64
 4   Employment                   888 non-null    float64
 5   Location Quotient            888 non-null    float64
 6   Percent of Total             814 non-null    float64
 7   Year and Quarter             888 non-null    object 
 8   GEO_ID                       816 non-null    object 
 9   Source                       888 non-null    object 
dtypes: float64(5), object(5)
memory usage: 69.5+ KB


In [106]:
#export to the SQLite database as only the JobsEQ quarterly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_IndustryEmployment_3YROutlook', conn, if_exists = 'replace', index = False)

888